This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np



In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = r"C:\Users\USER\25-26\CS 3B\JETHRO_RENDON\animals\training_set"  # e.g. './muffin-vs-chihuahua/train'
test_dir  = r"C:\Users\USER\25-26\CS 3B\JETHRO_RENDON\animals\test_set"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 6431 images belonging to 2 classes.
Found 1607 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION, DROPOUT, AND BATCH NORMALIZATION

from tensorflow.keras import layers, models, regularizers, optimizers

# Initial learning rate
initial_learning_rate = 0.001

# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = optimizers.Adam(learning_rate=lr_schedule)

# Build the model
# Applied dropout layers, L2 regularization, and batch normalization to reduce overfitting
# L2 regularization helps prevent overfitting by penalizing large weights
# BatchNormalization helps stabilize and speed up training
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),  # Input layer with image size
    
    # Conv Block 1
    layers.Conv2D(32, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),  # L2 regularization
    layers.BatchNormalization(),  # Normalize activations
    layers.MaxPooling2D(2,2),     # Downsample
    layers.Dropout(0.25),         # Dropout to prevent overfitting
    
    # Conv Block 2
    layers.Conv2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),
    
    # Conv Block 3
    layers.Conv2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),
    
    # Use GlobalAveragePooling2D instead of Flatten to reduce parameters
    layers.GlobalAveragePooling2D(),
    
    # Dense layer for learning high-level features
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    
    # Output layer with sigmoid for binary classification (Cats vs Dogs)
    layers.Dense(1, activation='sigmoid')
])

# Compile the model with Adam optimizer and binary crossentropy loss
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [6]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10 ,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE
)

Epoch 1/10
160/201 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.5766 - loss: 0.9490

In [ ]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"\n{'='*50}")
print(f"IMPROVED MODEL - Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'='*50}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 148ms/step - accuracy: 0.5325 - loss: 1.2883

IMPROVED MODEL - Test Accuracy: 0.5325 (53.25%)
Test Loss: 1.2883


In [ ]:
# SAVE THE IMPROVED MODEL
model.save('dogs_vs_cats_custom_rendon.h5')
print("Improved model saved as 'dogs_vs_cats_custom_rendon.h5'")

Improved model saved as 'dogs_vs_cats_custom_rendon.h5'


In [ ]:
# INFERENCE SCRIPT FOR IMPROVED MODEL
from tensorflow.keras.preprocessing import image

# Load model once for efficiency
_loaded_model = None

def predict_image(img_path, model_path='dogs_vs_cats_custom_rendon.h5', img_size=IMG_SIZE):
    global _loaded_model
    if _loaded_model is None:
        _loaded_model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = _loaded_model.predict(img_array, verbose=0)[0,0]
    label = "Cats" if pred >= 0.5 else "Dogs"
    confidence = pred if pred >= 0.5 else (1 - pred)
    print(f"Image: {img_path}")
    print(f"Prediction: {label}")
    print(f"Confidence: {confidence:.4f} ({confidence*100:.2f}%)")
    print(f"Raw prediction value: {pred:.4f}")
    print("-" * 50)
    return label, pred, confidence


In [ ]:
# Example usage:
predict_image(r"C:/Users/USER/25-26/CS 3B/JETHRO_RENDON/run_animals/animal_4.jpg")
predict_image(r"C:/Users/USER/25-26/CS 3B/JETHRO_RENDON/run_animals/animal_3.jpg")





Image: C:/Users/USER/25-26/CS 3B/JETHRO_RENDON/run_animals/animal_4.jpg
Prediction: Cats
Confidence: 0.9888 (98.88%)
Raw prediction value: 0.9888
--------------------------------------------------
Image: C:/Users/USER/25-26/CS 3B/JETHRO_RENDON/run_animals/animal_3.jpg
Prediction: Cats
Confidence: 0.9838 (98.38%)
Raw prediction value: 0.9838
--------------------------------------------------


('Cats', 0.9837506, 0.9837506)